# GDC

In [1]:
#%%
import pandas as pd
import glob
import os

sample_sheet = pd.read_csv('D:/data/RNASeq/GDC/sample_sheet.tsv', sep='\t')
clinical = pd.read_csv('D:/data/RNASeq/GDC/clinical.tsv', sep='\t')

csv_name = "gdcdataset_test"
csv_path = "D:/data/RNASeq/dataset/"

paths = glob.glob('D:\\data\\RNASeq\\GDC\\gdc_download_20220707_111311.305944\\*')

#iia = ['Stage I','Stage IA', 'Stage IB', 'Stage IIA']

for SPLIT in range(10):
    labels = []
    runs = []
    paths_array = []
    sample_ids = []
    case_ids = []

    case_id = open(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\splits\\testsplit{SPLIT}.txt", "r").read().split()
    split_file_ids = list(sample_sheet[sample_sheet["Case ID"].isin(case_id)]["File ID"])
    for path in paths:
        file_id = os.path.split(path)[-1]
        if file_id in split_file_ids:
            if not os.path.exists(path) or not os.path.isdir(path) or len(os.listdir(path)) == 0:
                print("Path does not exist or empty ", path)
                pass
            else:
                for file in glob.glob(path + r"\*"+".tsv"):
                    case_id = sample_sheet[sample_sheet["File ID"] == file_id]["Case ID"].iloc[0]
                    file_name = os.path.split(file)[-1]
                    paths_array.append(path)
                    runs.append(file_name)
                    labels.append(sample_sheet[sample_sheet["File ID"] == file_id]["Sample Type"].iloc[0])
                    sample_ids.append(sample_sheet[sample_sheet["File ID"] == file_id]["Sample ID"].iloc[0])
                    case_ids.append(case_id)

                    """
                    stage = clinical[clinical["case_submitter_id"] == case_id]["ajcc_pathologic_stage"].iloc[0]
                    if stage in iia:
                        labels.append("iia")
                    else:
                        labels.append("iib")
                    """

    # create dataset for knowseq
    mergedCounts = pd.DataFrame()
    mergedCounts['Path'] = paths_array
    mergedCounts['Run'] = runs
    mergedCounts['Class'] = labels
    mergedCounts['Sample ID'] = sample_ids
    mergedCounts['Case ID'] = case_ids

    mergedCounts.to_csv(csv_path+"labels/"+csv_name+ str(SPLIT) +'.csv', sep=',', index=False)

In [25]:
def clean_data(countsMatrix_train):
    for row in countsMatrix_train.iterrows():
        not_expressed = all(v == 0 for v in list(row[1]))
        if "PAR_Y" in row[0] and not_expressed:
            countsMatrix_train.drop(index=row[0], inplace=True)

    countsMatrix_train.index = [index.split(".")[0] for index in countsMatrix_train.index]
    #dataGtex_split.index = [index.split(".")[0] for index in dataGtex_split.index]

    countsMatrix_train = pd.concat([countsMatrix_train], axis=1)#, dataGtex_split], axis=1) # Concatenating all GTEx and GDC data for the split


    countsMatrix_train.fillna(0, inplace=True)

    return countsMatrix_train

In [5]:
# Knowseq does not work with STAR count files from GDC, therefore a full matrix must be created 
# COLUMNS=> SAMPLE_ID ROW=> GEN_ID The labels can be taken using R from the given .csv file
# lo que devuelve Knoqseq son dos columnas $countsMatrix (X) y $labels(y)

n_splits = 10

dataGtex = pd.read_csv("D:/data/RNASeq/GTEx/gene_reads_pancreas.gct", sep="\t", header=2)

dataGtex.index = dataGtex["Name"]
dataGtex.drop(columns=["id","Description", "Name"], inplace=True)

for SPLIT in range(n_splits):
    print("Split: ", SPLIT)
    countsMatrix = []
    gdcDataset=[]
    
    TRAIN_SPLITS = list(range(n_splits))
    # We take out the SPLIT and SPLIT+1 sets for val and testing
    if SPLIT == n_splits-1: # For the final split for validation we take the first one for test
        TRAIN_SPLITS.remove(0) 
    else:
        TRAIN_SPLITS.remove(SPLIT+1)
    TRAIN_SPLITS.remove(SPLIT)

    for TRAIN_SPLIT in TRAIN_SPLITS:
        mergedCounts = pd.read_csv(csv_path+"labels/"+csv_name+str(TRAIN_SPLIT)+'.csv', sep=',')
        gdcDataset.append(mergedCounts)

        for sample_id, path, name, label in zip(mergedCounts['Sample ID'], mergedCounts['Path'], mergedCounts['Run'], mergedCounts['Class']):
            counts = pd.read_csv(path+"\\"+name, sep='\t', header=1)#["unstranded"]
            counts = counts.set_index("gene_id")["unstranded"]
            counts.name = sample_id
            countsMatrix.append(counts)

    gdcDataset_train = pd.concat(gdcDataset) # All GDC labels
    countsMatrix_train = pd.concat(countsMatrix, axis=1) # All GDC splits are concatenated

    # TEST

    gdcDataset_test = pd.read_csv(csv_path+"labels/"+csv_name+str(SPLIT)+'.csv', sep=',')
    countsMatrix = []

    for sample_id, path, name, label in zip(gdcDataset_test['Sample ID'], gdcDataset_test['Path'], gdcDataset_test['Run'], gdcDataset_test['Class']):
            counts = pd.read_csv(path+"\\"+name, sep='\t', header=1)#["unstranded"]
            counts = counts.set_index("gene_id")["unstranded"]
            counts.name = sample_id
            countsMatrix.append(counts)

    countsMatrix_test = pd.concat(countsMatrix, axis=1) # All GDC splits are concatenated

    """    
    # Selecting all samples that have a Case ID in belonging to the corresponding split
    case_id = open(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\splits\\trainsplit{SPLIT}.txt", "r").read().split()
    split_sample_ids = [sample_id for sample_id in dataGtex.columns if ("-".join(sample_id.split("-")[:2]) in case_id)]
    dataGtex_split = dataGtex[split_sample_ids]

    gtexDataset = pd.DataFrame({"Case ID": ["-".join(sample_id.split("-")[:2]) for sample_id in list(dataGtex_split.columns)],
                                "Sample ID":list(dataGtex_split.columns), 
                                "Class":["Solid Tissue Normal" for i in dataGtex_split.columns]})

    #data.to_csv(csv_path + 'countsMatrixGTEx.csv', sep=',')

    #Deleting double (sexual) genes that are not expressed
    for row in dataGtex_split.iterrows():
        not_expressed = all(v == 0 for v in list(row[1]))
        if "PAR_Y" in row[0] and not_expressed:
            dataGtex_split.drop(index=row[0], inplace=True)
    """

    countsMatrix_train = clean_data(countsMatrix_train)
    countsMatrix_test = clean_data(countsMatrix_test)


    gdcDataset_train.to_csv(csv_path+'labels/gdcdataset_train'+str(SPLIT)+'.csv', sep =',')
    gdcDataset_test.to_csv(csv_path+'labels/gdcdataset_test'+str(SPLIT)+'.csv', sep =',')

    #gtexDataset.to_csv(csv_path+'labels/gtexdataset'+str(SPLIT)+'.csv', sep =',')
    
    countsMatrix_train.to_csv(csv_path+'countsMatrix/countsMatrixGDC_train_split'+str(SPLIT)+'.csv', sep=',')
    countsMatrix_test.to_csv(csv_path+'countsMatrix/countsMatrixGDC_test_split'+str(SPLIT)+'.csv', sep=',')

    #countsMatrix.to_csv(csv_path+'countsMatrix/countsMatrixGDC+GTEX_split'+str(SPLIT)+'.csv', sep=',')


Split:  0
Split:  1
Split:  2
Split:  3
Split:  4
Split:  5
Split:  6
Split:  7
Split:  8
Split:  9


# TCIA

In [28]:
#%%
import pandas as pd
import glob
import os

sample_sheet = pd.read_csv('D:/data/RNASeq/PDAC/sample_sheet.tsv', sep='\t')
clinical = pd.read_csv('D:/data/RNASeq/PDAC/clinical.tsv', sep='\t')

sample_sheet["Sample ID"] = sample_sheet["Sample ID"].str.split(",")
sample_sheet["Case ID"] = sample_sheet["Case ID"].str.split(",")
sample_sheet["Sample Type"] = sample_sheet["Sample Type"].str.split(",")
sample_sheet = sample_sheet.explode("Sample ID").explode("Case ID").explode("Sample Type")

csv_name = "pdac_dataset_test"
csv_path = "D:/data/RNASeq/dataset/"

paths = glob.glob('D:\\data\\RNASeq\\PDAC\\gdc_download_20220913_143316.680608\\*')

for SPLIT in range(10):
    labels = []
    runs = []
    paths_array = []
    sample_ids = []
    case_ids = []

    case_id = open(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\splits\\tcia_testsplit{SPLIT}.txt", "r").read().split()
    split_file_ids = list(sample_sheet[sample_sheet["Case ID"].isin(case_id)]["File ID"])

    for path in paths:
        file_id = os.path.split(path)[-1]
        if file_id in split_file_ids:
            if not os.path.exists(path) or not os.path.isdir(path) or len(os.listdir(path)) == 0:
                print("Path does not exist or empty ", path)
                pass
            else:
                for file in glob.glob(path + r"\*"+".tsv"):
                    case_id = sample_sheet[sample_sheet["File ID"] == file_id]["Case ID"].iloc[0]
                    file_name = os.path.split(file)[-1]
                    paths_array.append(path)
                    runs.append(file_name)
                    labels.append(sample_sheet[sample_sheet["File ID"] == file_id]["Sample Type"].iloc[0])
                    sample_ids.append(sample_sheet[sample_sheet["File ID"] == file_id]["Sample ID"].iloc[0])
                    case_ids.append(case_id)

    # create dataset for knowseq
    mergedCounts = pd.DataFrame()
    mergedCounts['Path'] = paths_array
    mergedCounts['Run'] = runs
    mergedCounts['Class'] = labels
    mergedCounts['Sample ID'] = sample_ids
    mergedCounts['Case ID'] = case_ids

    mergedCounts.to_csv(csv_path+"labels/"+csv_name+ str(SPLIT) +'.csv', sep=',', index=False)

In [29]:
# Knowseq does not work with STAR count files from GDC, therefore a full matrix must be created 
# COLUMNS=> SAMPLE_ID ROW=> GEN_ID The labels can be taken using R from the given .csv file
# lo que devuelve Knoqseq son dos columnas $countsMatrix (X) y $labels(y)

n_splits = 10

for SPLIT in range(n_splits):
    print("Split: ", SPLIT)
    countsMatrix = []
    gdcDataset=[]
    
    TRAIN_SPLITS = list(range(n_splits))
    # We take out the SPLIT and SPLIT+1 sets for val and testing
    if SPLIT == n_splits-1: # For the final split for validation we take the first one for test
        TRAIN_SPLITS.remove(0) 
    else:
        TRAIN_SPLITS.remove(SPLIT+1)
    TRAIN_SPLITS.remove(SPLIT)

    for TRAIN_SPLIT in TRAIN_SPLITS:
        mergedCounts = pd.read_csv(csv_path+"labels/"+csv_name+str(TRAIN_SPLIT)+'.csv', sep=',')
        gdcDataset.append(mergedCounts)

        for sample_id, path, name, label in zip(mergedCounts['Sample ID'], mergedCounts['Path'], mergedCounts['Run'], mergedCounts['Class']):
            counts = pd.read_csv(path+"\\"+name, sep='\t', header=1)#["unstranded"]
            counts = counts.set_index("gene_id")["unstranded"]
            counts.name = sample_id
            countsMatrix.append(counts)

    gdcDataset_train = pd.concat(gdcDataset) # All GDC labels
    countsMatrix_train = pd.concat(countsMatrix, axis=1) # All GDC splits are concatenated

    # TEST

    gdcDataset_test = pd.read_csv(csv_path+"labels/"+csv_name+str(SPLIT)+'.csv', sep=',')
    countsMatrix = []

    for sample_id, path, name, label in zip(gdcDataset_test['Sample ID'], gdcDataset_test['Path'], gdcDataset_test['Run'], gdcDataset_test['Class']):
            counts = pd.read_csv(path+"\\"+name, sep='\t', header=1)#["unstranded"]
            counts = counts.set_index("gene_id")["unstranded"]
            counts.name = sample_id
            countsMatrix.append(counts)

    countsMatrix_test = pd.concat(countsMatrix, axis=1) # All GDC splits are concatenated

    countsMatrix_train = clean_data(countsMatrix_train)
    countsMatrix_test = clean_data(countsMatrix_test)

    gdcDataset_train.to_csv(csv_path+'labels/TCIAdataset_train'+str(SPLIT)+'.csv', sep =',')
    gdcDataset_test.to_csv(csv_path+'labels/TCIAdataset_test'+str(SPLIT)+'.csv', sep =',')

    #gtexDataset.to_csv(csv_path+'labels/gtexdataset'+str(SPLIT)+'.csv', sep =',')
    
    countsMatrix_train.to_csv(csv_path+'countsMatrix/countsMatrixTCIA_train_split'+str(SPLIT)+'.csv', sep=',')
    countsMatrix_test.to_csv(csv_path+'countsMatrix/countsMatrixTCIA_test_split'+str(SPLIT)+'.csv', sep=',')

    #countsMatrix.to_csv(csv_path+'countsMatrix/countsMatrixGDC+GTEX_split'+str(SPLIT)+'.csv', sep=',')


Split:  0
Split:  1
Split:  2
Split:  3
Split:  4
Split:  5
Split:  6
Split:  7
Split:  8
Split:  9


In [1]:
n_splits = 10

for SPLIT in range(n_splits):
    print("Split: ", SPLIT)
    
    TRAIN_SPLITS = list(range(n_splits))
    # We take out the SPLIT and SPLIT+1 sets for val and testing
    if SPLIT == n_splits-1: # For the final split for validation we take the first one for test
        TRAIN_SPLITS.remove(0) 
    else:
        TRAIN_SPLITS.remove(SPLIT+1)
    TRAIN_SPLITS.remove(SPLIT)

    for TRAIN_SPLIT in TRAIN_SPLITS:
        print(TRAIN_SPLIT)




Split:  0
2
3
4
5
6
7
8
9
Split:  1
0
3
4
5
6
7
8
9
Split:  2
0
1
4
5
6
7
8
9
Split:  3
0
1
2
5
6
7
8
9
Split:  4
0
1
2
3
6
7
8
9
Split:  5
0
1
2
3
4
7
8
9
Split:  6
0
1
2
3
4
5
8
9
Split:  7
0
1
2
3
4
5
6
9
Split:  8
0
1
2
3
4
5
6
7
Split:  9
1
2
3
4
5
6
7
8
